In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes sentencepiece evaluate rouge_score bert-score

In [ ]:
# !pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
# !pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
!pip install -qU faker==35.2.0
!pip install -qU vllm==0.7.2

## login to huggingface_hub and  wandb

In [ ]:
from google.colab import drive

from huggingface_hub import login
import wandb
from google.colab import userdata

drive.mount('/content/drive')
wandb.login(key=userdata.get('WANDB_API_KEY'))
login(token=userdata.get('huggingface_token'), add_to_git_credential=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: omaryaser311 (omaryaser311-mansoura-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## import model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"

bbc = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_type=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    quantization_config=bbc,
    low_cpu_mem_usage=True,
).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


import needed libriries

In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

In [ ]:
story = """
ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
 حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
 تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

 الأبعاد الثلاثة للعلاقة بالمال
بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
 البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
 النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
 أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

الاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته بقدرته على توفير
المتعة والراحة. ومع ذلك، قد يصبح
البعض مدمنا على الإنفاق، في حين يتجه آخرون إلى التقشف المفرط خوفا من المستقبل.

الإدارة (M): أصحاب هذا النمط يعتبرون المال مسؤولية تتطلب التخطيط الدقيق. لكن في بعض الحالات،
 قد يتحول الأمر إلى هوس مفرط بإدارة الإنفاق، مما يؤثر سلبا على العلاقات الشخصية.

 كيف تؤثر العائلة على علاقتنا بالمال؟
يشير التقرير إلى أن التجارب الأسرية تلعب دورا رئيسيا في تحديد
 "الشخصية المالية" لكل فرد، على سبيل المثال، إذا كان أحد الوالدين يعتمد على المال
كمكافأة للسلوك الجيد، فقد يتبنى الطفل لاحقا النمط نفسه في حياته البالغة.

لتحليل هذه التأثيرات بشكل دقيق، طورت رابطة العلاج المالي
(Financial Therapy Association) أداة تسمى مخطط الجينوم المالي (Money Genogram)،
وهو نموذج يُستخدم لتحديد الأنماط المالية داخل العائلة.

تتضمن هذه الأداة:

رسم شجرة عائلية.
تصنيف أفراد العائلة وفقا للأبعاد الثلاثة للعلاقة بالمال (A ،U ،M).
تحديد ما إذا كان السلوك المالي لكل فرد صحيا (+) أو غير صحي (-).
على سبيل المثال، إذا نشأ شخص في عائلة
اعتادت على الإنفاق المفرط، فقد يكون لديه ميل قوي إلى اتباع النمط نفسه،
 أو العكس تماما، حيث يصبح مقتصدا بشكل مبالغ فيه كرد فعل نفسي.
"""

In [ ]:
StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified"
]

EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
    entity_value: str = Field(..., description="The actual name or value of the entity.")
    entity_type: EntityType = Field(..., description="The type of recognized entity.")


class extract_details(BaseModel):
    story_title : str = Field(... , min_length=15 , max_length=250 ,
                              description="a sutiable title for the story")
    story_keywords : List[str]  = Field(... , min_length=20 ,
                                        description="Relevant keywords associated with the story")
    story_summery : List[str] = Field(... , min_items = 1 , max_items = 5 ,
                                      description="Summarized key points about the story (1-5 points).")
    story_category : StoryCategory = Field(... , description="Category of the news story.")
    story_entites : List[Entity] = Field(..., min_items=1, max_items=10,
                                        description="List of identified entities in the story.")


/tmp/ipython-input-29985894.py:22: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  story_summery : List[str] = Field(... , min_items = 1 , max_items = 5 ,
/tmp/ipython-input-29985894.py:22: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  story_summery : List[str] = Field(... , min_items = 1 , max_items = 5 ,
/tmp/ipython-input-29985894.py:25: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  story_entites : List[Entity] = Field(..., min_items=1, 

Get Format Instrustions

In [ ]:

from langchain_core.output_parsers import JsonOutputParser

format_instructions = JsonOutputParser(pydantic_object=extract_details).get_format_instructions()
print(format_instructions)

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


Built The details_extraction_messages

In [ ]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            format_instructions ,
            "",

            "## Story Details:",
            "```json"
        ])
    }
]

In [ ]:
details_extraction_messages

[{'role': 'system',
  'content': 'You are an NLP data paraser.\nYou will be provided by an Arabic text associated with a Pydantic scheme.\nGenerate the ouptut in the same story language.\nYou have to extract JSON details from text according the Pydantic details.\nExtract details as mentioned in text.\nDo not generate any introduction or conclusion.'},
 {'role': 'user',
  'content': '## Story:\nذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،\n حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.\n\nالتقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول\nالرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة\n تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.\n\n الأبعاد الثلاثة للعلاقة بالمال\nبحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:\n\nالاكتساب (A): يميل الأفراد الذين ينتمون لهذا\n البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون\nفي تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا\n النمط هو إمكا

Every Model has It's Own chat_template Before tokinze the details_extraction_messages we sirst should apply_chat_template

In [ ]:
text = tokenizer.apply_chat_template(
    details_extraction_messages,#
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
text

'<|im_start|>system\nYou are an NLP data paraser.\nYou will be provided by an Arabic text associated with a Pydantic scheme.\nGenerate the ouptut in the same story language.\nYou have to extract JSON details from text according the Pydantic details.\nExtract details as mentioned in text.\nDo not generate any introduction or conclusion.<|im_end|>\n<|im_start|>user\n## Story:\nذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،\n حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.\n\nالتقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول\nالرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة\n تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.\n\n الأبعاد الثلاثة للعلاقة بالمال\nبحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:\n\nالاكتساب (A): يميل الأفراد الذين ينتمون لهذا\n البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون\nفي تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا\n النمط هو إمكانية التحول إلى هوس 

Tokinize Text

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Return Outpot

In [ ]:
generated_ids = [
    output_ids[len(input_ids):] # بما ان بيرجع الانبوت مع الاوبتوبت فان اعاوز ابدا بالاوبوت بس اعرضه ف ببدا من نهايه الانبوت لان بعدها هيتكتب الابتوبت
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
] #generated_ids بيرجع الانبوت مع الاوبتبوت يعني البرومبت مع الناتج ليه او الرد بس علي شكل ايديهات علشان كده استخدمنا model_inputs.input_ids

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] # skip_special_tokens=True دا لان المودل بسبب التشات تمبليت بيضيف توكنز خاص بيه زي ايم استرت واسسيسمنت وهكذا فانا مش عاوزها

In [ ]:
# we wanted the details to be in the same lang of the story we told the model that but it answer with diffrent lang
print(response)

```json
{
  "story_title": "The Role of Family in Financial Relationships",
  "story_keywords": [
    "family influence on money",
    "financial relationships",
    "inheritance patterns",
    "personal finance",
    "economic impact",
    "parental influence",
    "child development",
    "wealth management",
    "social dynamics"
  ],
  "story_summery": [
    "Families play a crucial role in shaping individual financial behaviors and attitudes.",
    "Inherited financial habits can significantly affect how individuals approach wealth and money throughout their lives.",
    "Understanding these inherited traits helps in developing effective strategies for managing personal finances."
  ],
  "story_category": "economy",
  "story_entites": [
    {
      "entity_value": "Financial Therapy Association",
      "entity_type": "organization"
    },
    {
      "entity_value": "Shawn Ennett",
      "entity_type": "person"
    }
  ]
}
```


Task 2 : Translation Of Story

In [ ]:
class TranslatedStory(BaseModel):
    translated_title: str = Field(..., min_length=5, max_length=300,
                                  description="Suggested translated title of the news story.")
    translated_content: str = Field(..., min_length=5,
                                    description="Translated content of the news story.")

#  format_instructions of TranslatedStory
format_instructions_for_TranslatedStory = JsonOutputParser(pydantic_object=TranslatedStory).get_format_instructions()
print(format_instructions_for_TranslatedStory)

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


Built Translation Message

In [ ]:
Targeted_Language = "French"
translation_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",
            "## Targeted Language" ,
            Targeted_Language ,
            "",

            "## Pydantic Details:",
            format_instructions_for_TranslatedStory ,
            "",

            "## Story Details:",
            "```json"
        ])
    }]

apply_chat_template on  translation_messages

In [ ]:
text = tokenizer.apply_chat_template(
    translation_messages ,
    tokenize=False, # telling the llm not to tokinze it to numbers now
    add_generation_prompt=True # adds <|im_start|>assistant at the end to notice the model to satrt answer
)

In [ ]:
print(text)

<|im_start|>system
You are a professional translator.
You will be provided by an Arabic text.
You have to translate the text into the `Targeted Language`.
Follow the provided Scheme to generate a JSON
Do not generate any introduction or conclusion.<|im_end|>
<|im_start|>user
## Story:
ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
 حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
 تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

 الأبعاد الثلاثة للعلاقة بالمال
بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
 البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
 النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
 أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

الاستخدام (U): يرى هؤلاء الأشخاص الما

tokinze text & Generate Answer

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)
# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] /

In [ ]:
generated_ids = [
    output_ids[len(input_ids):] # بما ان بيرجع الانبوت مع الاوبتوبت فان اعاوز ابدا بالاوبوت بس اعرضه ف ببدا من نهايه الانبوت لان بعدها هيتكتب الابتوبت
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
] #generated_ids بيرجع الانبوت مع الاوبتبوت يعني البرومبت مع الناتج ليه او الرد بس علي شكل ايديهات علشان كده استخدمنا model_inputs.input_ids

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

```json
{
  "translated_title": "La famille joue un rôle crucial dans la formation des relations financières",
  "translated_content": "Une étude de l'Université de Stanford a montré que chaque individu possède une «personnalité financière» déterminée par sa façon d'échanger avec les fonds, qui est influencée directement par le mode de vie et la culture familiale."
}


Evaluate LLama3.1

## Using Chat Model + LangChain

In [ ]:
import torch
from langchain_community.chat_models import ChatOllama
device = "cuda" if torch.cuda.is_available() else "cpu"
llm = ChatOllama(model="llama3.1:latest")

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.chat_models import ChatOllama


format_instructions = JsonOutputParser(pydantic_object=extract_details).get_format_instructions()

prompt = f"""
You are an NLP data parser.
You will be provided with an Arabic story and a Pydantic schema.
Generate the output in the same language as the story.
Extract JSON details from the story according to the schema.
Only extract what is mentioned explicitly.
Do not generate any introduction or conclusion.

## Story:
{story.strip()}

## Pydantic Details:
{format_instructions}

## Story Details:
```json
"""

response = llm.invoke(prompt)
print(response.content)

llama3 gives us the result that we want

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

templete = ChatPromptTemplate.from_messages([
    ("system", '''You are an NLP data parser. \
    You will be provided with an Arabic story and a Pydantic schema. \
    Generate the output in the same language as the story. \
    Extract JSON details from the story according to the schema. \
    Only extract what is mentioned explicitly. \
    Do not generate any introduction or conclusion.'''),

    ("human", '''## Story:
    {story}

    ## Pydantic Details:
    {format_instructions}

    ## Story Details:
    ```json
    ''')
])


chain = templete | llm | JsonOutputParser(pydantic_object=extract_details)
response = chain.invoke({
    "story" :story.strip(),
    "format_instructions" : format_instructions
})
print(response)

In [ ]:
# llama 3 on Translation Task
from langchain_core.prompts import ChatPromptTemplate

templete = ChatPromptTemplate.from_messages([
    ("system", '''You are a professional translator. \
    You will be provided by an Arabic text. \
    You have to translate the text into the `Targeted Language`. \
    Follow the provided Scheme to generate a JSON. \
    Do not generate any introduction or conclusion.'''),

    ("human", '''## Story:
    {story}
    ## Targeted Language:
    {Targeted_Language}

    ## Pydantic Details:
    {format_instructions}

    ## Story Details:
    ```json
    ''')
])


chain = templete | llm | JsonOutputParser(pydantic_object=extract_details)
response = chain.invoke({
    "story" :story.strip(),
    "Targeted_Language" : "English" ,
    "format_instructions" : format_instructions_for_TranslatedStory
})
print(response)

## Knowledge Distillation

Load Stories

In [ ]:
# import rand.
data = []
with open('./Stories/news-sample.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():  # تجاهل الأسطر الفارغة
            data.append(json.loads(line))

random.Random(101).shuffle(data) # do shuffle because sometimes data are arranged like all sports then all politics and so on

print(f"Loaded {len(data)} records.")
print(data[0])

In [ ]:
print(data[0]['content'])

In [ ]:
save_to = join("Stories", "sft.jsonl")

for story in tqdm(data) :
    templete = ChatPromptTemplate.from_messages([
    ("system", '''You are an NLP data parser. \
    You will be provided with an Arabic story and a Pydantic schema. \
    Generate the output in the same language as the story. \
    Extract JSON details from the story according to the schema. \
    Only extract what is mentioned explicitly. \
    Do not generate any introduction or conclusion.'''),

    ("human", '''## Story:
    {story}

    ## Pydantic Details:
    {format_instructions}

    ## Story Details:
    ```json
    ''')
    ])

    chain = templete | llm | JsonOutputParser(pydantic_object=extract_details)
    response = chain.invoke({
        "story" :story.strip(),
        "format_instructions" : format_instructions
    })

    with open(save_to , 'a' ,encoding="utf-8") as saved_json :
        saved_json.write(json.dumps({
            "story": story['content'].strip(),
            "task": "Extrat the story details into a JSON.",
            "output_scheme": format_instructions,
            "response": response,
        }, ensure_ascii=False, default=str)  + "\n" )



In [ ]:
# For Translation

save_to = join("Stories", "sft.jsonl")

for story in tqdm(data) :
    templete = ChatPromptTemplate.from_messages([
        ("system", '''You are a professional translator. \
        You will be provided by an Arabic text. \
        You have to translate the text into the `Targeted Language`. \
        Follow the provided Scheme to generate a JSON. \
        Do not generate any introduction or conclusion.'''),

        ("human", '''## Story:
    {story}
    ## Targeted Language:
    {Targeted_Language}

    ## Pydantic Details:
    {format_instructions}

    ## Story Details:
    ```json
    ''')
    ])

    chain = templete | llm | JsonOutputParser(pydantic_object=TranslatedStory)
    response = chain.invoke({
        "story" :story.strip(),
        "Targeted_Language" : "English" ,
        "format_instructions" : format_instructions_for_TranslatedStory
    })


    with open(save_to , 'a' ,encoding="utf-8") as saved_json :
        saved_json.write(json.dumps({
            "story": story['content'].strip(),
            "task": "Translate the story Content into another Langauge.",
            "output_scheme": format_instructions_for_TranslatedStory,
            "response": response,
        }, ensure_ascii=False, default=str)  + "\n" )


## Format Finetuning Datasets

In [ ]:

!unrar x "/content/drive/MyDrive/fine_tunning_qwenn/train_ds/train.rar" "/content/data/"
!unrar x "/content/drive/MyDrive/fine_tunning_qwenn/val_ds/val.rar" "/content/data/"



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/fine_tunning_qwenn/train_ds/train.rar

Creating    /content/data                                             OK
Extracting  /content/data/train.json                                      75%100%  OK 
All OK

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/fine_tunning_qwenn/val_ds/val.rar

Extracting  /content/data/val.json                                       100%  OK 
All OK


In [ ]:

sft_data_path = join("./Stories", "sft.jsonl")
llm_finetunning_data = []

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

for line in open(sft_data_path , encoding='utf-8'):
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# Story:",
            rec["story"],

            "# Task:",
            rec["task"],

            "# Output Scheme:",
            rec["output_scheme"],
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),
            "```"
        ]),
        "history": []
    })

random.Random(101).shuffle(llm_finetunning_data)

In [ ]:
llm_finetunning_data[0]

In [ ]:
llm_finetunning_data[5]

In [ ]:
len(llm_finetunning_data)

In [ ]:
train_sample_sz = 2700

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join("Stories", "llamafactory-finetune-data"), exist_ok=True)

with open(join("Stories", "llamafactory-finetune-data", "train.json"), "w" , encoding='utf-8') as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join("Stories", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [ ]:
# افتح الملف
with open("/content/data/train.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
def tokenize_function(batch):
    prompts = [i + "\n" + j for i, j in zip(batch["instruction"], batch.get("input", [""]*len(batch["instruction"])))]

    # tokenize inputs
    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=512,
        padding="max_length",
    )

    # tokenize outputs (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["output"],
            truncation=True,
            max_length=512,
            padding="max_length",
        )

    tokenized["labels"] = labels["input_ids"]
    return tokenized


In [ ]:
from datasets import Dataset

# حول list من dicts لـ Dataset
train_dataset = Dataset.from_list(data)
# val_dataset = Dataset.from_list(eval_ds)

# طبق tokenize
tokenized_train = train_dataset.map(tokenize_function, batched=True)
# tokenized_val = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Finetuning

## setup up lora_config

In [ ]:
from peft import LoraConfig , get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj",  "v_proj" ] ,
    lora_dropout=0.05 ,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model , lora_config)
model.print_trainable_parameters()

trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


## Tranning Lara

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./fine_tunned_qwen2.5_story_parser",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=12,
    learning_rate=2e-4,
    num_train_epochs=3,
    bf16=True,
    save_steps=200,
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train,
    data_collator=data_collator,
    args=training_args,
)

trainer.train()


Step,Training Loss


##Fine Tunning Using Lora Factory

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 462, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 462 (delta 120), reused 302 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (462/462), 5.16 MiB | 6.04 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 10.5 MB/s eta 0:00:00


In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
# ```
  #  "news_finetune_train": {
  #       "file_name": "/gdrive/MyDrive/youtube-resources/llm-finetuning/datasets/llamafactory-finetune-data/train.json",
  #       "columns": {
  #           "prompt": "instruction",
  #           "query": "input",
  #           "response": "output",
  #           "system": "system",
  #           "history": "history"
  #       }
  #   },
  #   "news_finetune_val": {
  #       "file_name": "/gdrive/MyDrive/youtube-resources/llm-finetuning/datasets/llamafactory-finetune-data/val.json",
  #       "columns": {
  #           "prompt": "instruction",
  #           "query": "input",
  #           "response": "output",
  #           "system": "system",
  #           "history": "history"
  #       }
  #   }
# ```

# https://wandb.ai/mr-bakrianoo/llamafactory/runs/apwbkni9
# https://wandb.ai/mr-bakrianoo/llamafactory/runs/c5tf0q90

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 32
lora_target: q_proj,v_proj

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
max_samples: 100
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /gdrive/MyDrive/youtube-resources/llm-finetuning/models/checkpoint-1500
output_dir: /content/drive/MyDrive/fine_tunning_qwenn
logging_steps: 10
save_steps: 25
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 50

report_to: wandb
run_name: newsx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "OmarFadlallah/news-analyzer"
hub_private_repo: true
hub_strategy: checkpoint


Overwriting /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

2025-12-08 20:37:41.512030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765226261.533716    5272 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765226261.539856    5272 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765226261.555442    5272 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765226261.555468    5272 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765226261.555472    5272 computation_placer.cc:177] computation placer alr

## Evaluation


In [ ]:
ft_model_id = "/content/drive/MyDrive/fine_tunning_qwenn"
model.load_adapter(ft_model_id)

In [ ]:
story = '''
تواصل أسواق الدواجن بالدقهلية عرض كميات متنوعة من الدواجن والطيور، في ظل استقرار حركة البيع والشراء خلال ساعات النهار.

ويظهر اهتمام واضح من المواطنين بمتابعة الأسعار اليومية، خاصة لكون الدواجن من السلع الأساسية داخل كل منزل، وتُعد من أكثر الخيارات التي يعتمد عليها الأهالي في وجباتهم اليومية. وتستقبل الأسواق كميات مناسبة من المزارع، ما يسهم في توفير الأصناف المطلوبة بمختلف أنواعها.

وتشهد الأسواق حركة معتدلة منذ بداية اليوم، مع استمرار توافد الأهالي لاختيار المنتجات المناسبة لهم. ويعتمد الكثير من المواطنين على متابعة حركة السوق اليومية، خصوصًا أن الدواجن تعد من أكثر السلع اعتمادًا داخل الوجبات المنزلية، نظرًا لتنوع طرق طهيها وسهولة تجهيزها.
لذا يفضل البعض متابعة الأسعار أولًا عبر المنصات الإخبارية قبل اتخاذ قرار الشراء.

أسعار الدواجن
رصدت الدستور الاسعار بالاسواق كالاتي: سجل سعر الدواجن البيضاء من 75 إلى 80 جنيهًا للكيلو، بينما بلغت الساسو 90 إلى 95 جنيهًا، والدواجن البلدي تراوحت بين 100 و115 جنيهًا للكيلو، أما الدواجن الأمهات فاستقرت أسعارها ما بين 65 إلى 70 جنيهًا، وسعر البط البلدي بلغ حوالي 120 جنيهًا للكيلو، والرومي من 140 حتى 160 جنيهًا.

وتراوحت أسعار البط كالآتي: البط البلدي: 140 جنيهًا للكيلو، البط الفرنسي: 90 جنيهًا للكيلو، البط المسكوفي: 120 جنيهًا للكيلو، البط البكيني: 100 جنيه للكيلو.

وسجلت أسعار الكتاكيت: الكتاكيت البيضاء: 45 - 60 جنيهًا للكتكوت، الكتاكيت الساسو: 30 - 40 جنيهًا للكتكوت.

بلغ سعر البانيه المجمد من 170 إلى 190 جنيهًا للكيلو، وشيش طاووق من 160 إلى 180 جنيهًا، والدبابيس من 110 إلى 130 جنيهًا، فيما تراوحت أسعار الأجنحة بين 70 و85 جنيهًا، والهياكل من 35 إلى 40 جنيهًا، والكبد والقوانص من 70 إلى 85 جنيهًا.

ذكرت منى عبد الهادي، ربة منزل من المنصورة، أنها تتابع أسعار الدواجن يوميًا لأنها جزء أساسي من وجبات أسرتها، مؤكدة أن توفر الأنواع المختلفة يساعدها في اختيار ما يناسب احتياجاتها الأسبوعية.

'''

In [ ]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            format_instructions ,
            "",

            "## Story Details:",
            "```json"
        ])
    }
]
details_extraction_messages

[{'role': 'system',
  'content': 'You are an NLP data paraser.\nYou will be provided by an Arabic text associated with a Pydantic scheme.\nGenerate the ouptut in the same story language.\nYou have to extract JSON details from text according the Pydantic details.\nExtract details as mentioned in text.\nDo not generate any introduction or conclusion.'},
 {'role': 'user',
  'content': '## Story:\nتواصل أسواق الدواجن بالدقهلية عرض كميات متنوعة من الدواجن والطيور، في ظل استقرار حركة البيع والشراء خلال ساعات النهار.\n\nويظهر اهتمام واضح من المواطنين بمتابعة الأسعار اليومية، خاصة لكون الدواجن من السلع الأساسية داخل كل منزل، وتُعد من أكثر الخيارات التي يعتمد عليها الأهالي في وجباتهم اليومية. وتستقبل الأسواق كميات مناسبة من المزارع، ما يسهم في توفير الأصناف المطلوبة بمختلف أنواعها.\n\nوتشهد الأسواق حركة معتدلة منذ بداية اليوم، مع استمرار توافد الأهالي لاختيار المنتجات المناسبة لهم. ويعتمد الكثير من المواطنين على متابعة حركة السوق اليومية، خصوصًا أن الدواجن تعد من أكثر السلع اعتمادًا داخل الوجبا

In [ ]:
def generate_responce(TEXT : list):
  text = tokenizer.apply_chat_template(
      TEXT ,
      tokenize=False,
      add_generation_prompt=True
  )

  model_inputs = tokenizer([text], return_tensors="pt").to(device)

  generated_ids = model.generate(
      model_inputs.input_ids ,
      max_new_tokens=1024,
      do_sample=False, top_k=None, temperature=None, top_p=None,
  )

  generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

response = generate_responce(details_extraction_messages)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print(response)

```json
{
  "story_title": "أسعار الدواجن بالدقهلية",
  "story_keywords": [
    "أسعار الدواجن",
    "الدقهلية",
    "البيع والشراء",
    "الدجاج",
    "البط",
    "الرومي",
    "البانيه المجمد",
    "الشيش الطاووق",
    "الدبابيس",
    "الأجنحة",
    "الكبد والقوانص"
  ],
  "story_summery": [
    "تواصل أسواق الدواجن بالدقهلية عرض كميات متنوعة من الدواجن والطيور.",
    "في ظل استقرار حركة البيع والشراء خلال ساعات النهار.",
    "يظهر اهتمام واضح من المواطنين بمتابعة الأسعار اليومية.",
    "الدواجن تعد من السلع الأساسية داخل كل منزل.",
    "تُعد من أكثر الخيارات التي يعتمد عليها الأهالي في وجباتهم اليومية.",
    "تستقبل الأسواق كميات مناسبة من المزارع.",
    "تشهد الأسواق حركة معتدلة منذ بداية اليوم.",
    "تعتمد الكثير من المواطنين على متابعة حركة السوق اليومية.",
    "أسعار الدواجن تتراوح بين 75 و80 جنيهًا للكيلو.",
    "البط البلدي: 140 جنيهًا للكيلو.",
    "الرومي: 140 حتى 160 جنيهًا.",
    "الهياكل: 35 إلى 40 جنيهًا."
  ],
  "story_category": "economy",
  "story_entites": [
    {
 

## Cost Estamation

In [ ]:
from tqdm.auto import tqdm
from faker import Faker
import random
from datetime import datetime

start_time = datetime.now()
fake = Faker('ar')

input_tokens = 0
outpot_tokens = 0

for i in tqdm(range(20)):
  prompt = fake.text(max_nb_chars=random.randint(200, 300))

  messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
  response = generate_responce(messages)

  input_tokens += len(tokenizer.apply_chat_template(messages))
  outpot_tokens += len(tokenizer.encode(response))


total_time = (datetime.now() - start_time).total_seconds()

print(f"Total Time: {total_time} seconds")
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {outpot_tokens}")
print(f"Total Tokens: {input_tokens + outpot_tokens}")

## VLLM

In [ ]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
ft_model_id = "/content/drive/MyDrive/fine_tunning_qwenn"


!nohup vllm serve "{base_model_id}" --dtype=half --gpu-memory-utilization 0.8 --max_lora_rank 64 --enable-lora --lora-modules news-lora="{ft_model_id}" &


nohup: appending output to 'nohup.out'


In [ ]:
!python --version


Python 3.12.12
